1. 연속형, 범주형 판단
2. small, big class에 따라 분리
3. 각 class에 따로 연속형만 GMM(k=2) fitting 후 상위 N % 만 Filtering
4. ctgan으로 연속형 변수만 생성 후 Density 기반의 적합성 판단
5. 생성된 연속형 변수와 cosine similarity 기반의 가장 가까운 기존 변수의 범주형값 카피

In [ ]:
!pip install ctgan
!pip install sdv

In [ ]:
from sdv.single_table import CTGANSynthesizer

In [ ]:
from sdv.datasets.demo import download_demo

data, metadata = download_demo(
    modality='single_table',
    dataset_name='fake_hotel_guests'
)

In [ ]:

synthesizer = CTGANSynthesizer(metadata)
synthesizer.fit(data)

synthetic_data = synthesizer.sample(num_rows=10)

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
# GMM: n_components = 모델의 총 수
gmm = GaussianMixture(n_components=3, random_state=0)
gmm.fit(iris.data)
gmm_cluster_labels = gmm.predict(iris.data)

In [ ]:
from sdv.tabular import CTGAN
from sdv.evaluation import evaluate
from sklearn.neighbors import KernelDensity
from sklearn.mixture import GaussianMixture
import numpy as np
import pandas as pd


batch_size = 5000
epochs = 100
model = CTGAN(batch_size=batch_size, epochs=epochs, verbose=True)
model.fit(train_df)
model.save('ctgan_30daysofml_model.pkl')

n_generated_data = 1000
generated_df = model.sample(n_generated_data)

score = evaluate(generated_df, train_df.sample(n_generated_data))
score

# ctgan
class Autoencoder(nn.Module):

  


# imbalanced에 data level로 해결하는 모델
class DDHS:
    
  #데이터를 KDE로 가우시안 분포를 이용하여 중간 %를 추출하는 함수
  # start, last에 퍼센트를 입력

  def extract_middle_percent(self,data, start, last):
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(data_scaled)
    
    log_prob = kde.score_samples(data_scaled)
    prob = np.exp(log_prob)
    threshold_low, threshold_high = np.percentile(prob, [start, last])
    mask = np.logical_and(prob >= threshold_low, prob <= threshold_high) #######
    data_middle = data[mask]

    if len(data_middle) > 0 :
      return data_middle
    else:  
      print("No middle 50% found, returning original data")
      return np.array([])

  #  각 feature 안의 값을 복원추출하는 함수

  def reconstruct_features(self,data):
    mean = data.mean(axis=0)
    std = data.std(axis=0)
    reconstructed = np.random.randn(*data.shape) * std + mean
    return reconstructed

  # synthetic sample을 생성하는 함수
  # 라벨과과 데이터를 하나의 데이터프레임으로 출력
  # small class / large class의 비율 = ratio 


  # 디폴트트 파라미터는 뒤로 빼서 입력하기
  def generate_synthetic_sample(self,X,Y, ratio=0.3):

    data = pd.concat([X,Y],axis=1)

    # small class
    data_A = data[data[Y.columns[0]]== Y.value_counts().idxmin()[0]  ].loc[:, data.columns != Y.columns[0]].astype(float).values

    # large class 
    data_B = data[data[Y.columns[0]]== Y.value_counts().idxmax()[0] ].loc[:, data.columns != Y.columns[0]].astype(float).values

    # autoencoder를 사용하여 잠재 변수를 추출
    with torch.no_grad():
        encoded_A, _ = self.model(torch.tensor(data_A).float())
        encoded_B, _ = self.model(torch.tensor(data_B).float())
  
    # Majority : 입력받은 퍼센트 보다 Density가 큰 샘플을 Keep→ 입력받은 퍼센트 샘플을 Classifier 의 Train 데이터로 활용
    # Minority : 입력받은 퍼센트 을 사용해서 더 높은 기준을 설정 → 입력받은 퍼센트 샘플을 Classifier 의 Train 데이터로 활용 → 25% 샘플을 Subsequence 생성 과정의 샘플로 활용
  
    encoded_A_middle = self.extract_middle_percent(encoded_A.cpu().numpy(),50 - self.small_percent/2 , 50+ self.small_percent/2) 
    encoded_B_middle = self.extract_middle_percent(encoded_B.cpu().numpy(),50 -self.large_percent/2, 50 + self.large_percent/2)

    # 중간 25%의 잠재 변수로부터 feature를 복원추출 
    reconstructed_features = self.reconstruct_features(self.extract_middle_percent(encoded_A.cpu().numpy(),37.5, 62.5))
    # 임의의 위치에 synthetic sample 생성
    center_A = np.mean(encoded_A.cpu().numpy(), axis=0, dtype=np.float64, out=None)

    center_B = np.mean(encoded_B.numpy(), axis=0, dtype=np.float64, out=None) 

    radius_A = np.max(np.linalg.norm(encoded_A.cpu().numpy() - center_A, axis=1))

    synthetic_sample = pd.DataFrame() # 최종 합치기
   
   # 합성된 개수 / 원래 클래스 개수
    while len(synthetic_sample)/len(data_A) >= ratio :
        z = np.random.randn(latent_dim)
        if np.linalg.norm(z - center_A) < np.linalg.norm(z - center_B) and np.linalg.norm(z - center_A) < radius_A:
            synthetic_sample.append(z) #, ignore_index=True)

    # 최종 출력할 데이터 
    encoded_B_middle = pd.DataFrame(encoded_B_middle)
    encoded_B_middle['label'] = Y.value_counts().idxmin()[0]

    encoded_A_middle = pd.DataFrame(encoded_A_middle)
    encoded_A_middle['label'] = Y.value_counts().idxmax()[0] 

    synthetic_sample['label'] = Y.value_counts().idxmax()[0] 

    ouput = pd.concat([encoded_B_middle,encoded_A_middle,synthetic_sample ] )

    x_ = ouput.loc[:, ouput.columns != 'label']
    x_.columns = X.columns
    y_ = ouput['label']
    y_.columns = Y.columns

    return x_ , y_

  def fit(self,X,Y,large_percent = 50 , small_percent = 75 ,lr = 1e-3 ,num_epochs = 50, ratio = 1):
    self.large_percent = large_percent
    self.small_percent = small_percent
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    input_dim = len(X.columns) # 데이터의 차원
    latent_dim = len(X.columns) # 잠재 변수의 차원
    #ratio =  합성된 데이터 수/small class  비율
    #large_percent : large class의 추출 비율
    #small_percent : small class의 추출 비율 
    #lr = 1e-3 # 학습률
    #num_epochs = 50 # 학습 에폭 수
    
    self.model = Autoencoder(input_dim, latent_dim).to(device)
    optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)


    for epoch in range(num_epochs):
      x = torch.tensor(X.to_numpy()).float()#.to(device)
      y = torch.tensor(Y[Y.columns[0]].to_numpy()).float().to(device)

      encoded, decoded = self.model(x)
      reconstruction_loss = F.mse_loss(decoded, x)
      center_loss = self.model.get_center_loss(encoded, y)

      loss = reconstruction_loss + center_loss
      cross_entropy_loss = F.cross_entropy(decoded, y.long()) # y를 long 형으로 요구
      loss += cross_entropy_loss

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    
    synthetic_X, synthetic_Y = self.generate_synthetic_sample(X,Y, ratio)
    return synthetic_X, synthetic_Y


  def __init__(self):
    self.result = 0




In [ ]:
cd

In [ ]:
import pandas as pd 
magic = pd.read_csv('/Users/jangsehwan/Documents/DDHS개발/magic04.data')

In [ ]:
magic.iloc[:, [1,4,6]]

In [ ]:
# 순차적으로 구조 점검 및 구현 
# 범주형 변수 찾아내는 부분 추가 
# ctgan, gmm 버전 오류해결 



In [ ]:
from sdv.single_table import CTGANSynthesizer
from sklearn.mixture import GaussianMixture
import numpy as np
import pandas as pd 


# imbalanced에 data level로 해결하는 모델
class figen:
    def fit(self,small_X,small_Y,large_X,large_Y,ratio,index):
        '''
            index : 범주형, 연속형 구분 인덱스
            small_X : small class의 x
            small_Y : small class의 y
            large_X : large class의 x
            large_Y : large class의 y
            ratio : small class+생성된 데이터와 large class의 비율 
            
            
        '''
        self.index = index
        self.ratio = ratio
        self.index = index 
    
        # 합성+ 기존 data set 생성 
        synthetic_X, synthetic_Y = self.generate_synthetic(small_X,large_Y,self.ratio, self.index)
        
        
        return synthetic_X, synthetic_Y
    
    
    def generate_synthetic(self, small_X, large_X, ratio, index ):
        '''
            small_X : 
            large_X : 
            ratio : small class와 large class 의 비율 
            index : 연속형 변수의 컬럼 인덱스
        '''
        
        # 연속형 변수만 가져오는 부분 
        continue_small_X = small_X.iloc[:,self.index]
        continue_large_X = large_X.iloc[:,self.index]
        
        
        # 상위 n% 필터링 부분 
        midlle_small_X = extract_middle_percent(self,continue_small_X, 25, 75) # 추후에 하이퍼 파라미터로 뺄 수 있음
        
        midlle_large_X = extract_middle_percent(self,continue_large_X, 15, 85) # 추후에 하이퍼 파라미터로 뺄 수 있음
                
        
        # ctgan으로 연속형 생성 부분 
        
        synthesizer.fit(midlle_small_X)
        
        synthetic_data = synthesizer.sample(num_rows=10) 
        
        # 데이터 적합 판단 
        
        suitable_generated_small_X = suitable_judge(synthetic_data)
        
        # 범주형 변수 값 찾아내기
        
        find_categorical()
        
        
        # small class와 large class 합치기
        
        return synthetic_X, synthetic_Y
        

    def extract_middle_percent(self,data, start, last):
        '''
            data : 입력 데이터
            start : 추출 시작 percentile 
            last : 추출 끝 percentile
        
            
        '''
        
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(data)
        
        kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(data_scaled) # 계산이 안터지도록 하기, gmm으로 변경 
    
        log_prob = kde.score_samples(data_scaled)
        prob = np.exp(log_prob)
        threshold_low, threshold_high = np.percentile(prob, [start, last])
        mask = np.logical_and(prob >= threshold_low, prob <= threshold_high) 
        data_middle = data[mask]

        if len(data_middle) > 0 :
            return data_middle
        else:  
            print("No middle 50% found, returning original data")
            return np.array([])

        
        
    def find_categorical():
        '''
        '''
    
    
    
    
    def suitable_judge(generated_x, small_X, large_X):
        '''
           generated_x : 생성된 small class x 데이터
           small_X : 원본 small class x 데이터
           large_X : 원본 large class x 데이터
        '''
        
        center_small_X = np.mean(small_X.cpu().numpy(), axis=0, dtype=np.float64, out=None)
        
        radius_small_X = np.max(np.linalg.norm(small_X.cpu().numpy() - center_small_X, axis=1))
        
        center_large_X = np.mean(large_X.cpu().numpy(), axis=0, dtype=np.float64, out=None)
        
        radius_large_X = np.max(np.linalg.norm(large_X.cpu().numpy() - center_large_X, axis=1))

        synthetic_sample = pd.DataFrame() # 최종 합치기
        
        # 합성된 개수 / 원래 클래스 개수 <= ratio
        while len(synthetic_sample)/len(large_X) >= ratio :
            
            z = generated_x # 얼마나 생성하는 부분 추가, 조건문으로 생성을 N개 후 len(synthetic_sample)/len(large_X) >= ratio 만족시 그만 생성하는 것으로
            
            # 생성된 small class 데이터가 small, large class 중 small에 가까운지, small class의 지름을 넘지는 않는지
            if np.linalg.norm(z - center_small_X) < np.linalg.norm(z - center_large_X ) and np.linalg.norm(z - center_small_X) < radius_small_X:
                synthetic_sample.append(z) #, ignore_index=True)
                
        return synthetic_sample
    
    
    def __init__(self):
        self.result = 0



